In [106]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [107]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report
print("Done!")

Done!


In [108]:
train=pd.read_csv("/kaggle/input/playground-series-s5e10/train.csv")
print("Read!")

Read!


In [109]:
Xtrain = train.drop(columns=['accident_risk'])
ytrain = train.accident_risk

In [110]:
X_train_split, X_valid, y_train_split, y_valid = train_test_split(
    Xtrain, ytrain, test_size=0.2, random_state=42
)

In [111]:
combined = pd.concat([X_train_split, X_valid], axis=0)
combined_encoded = pd.get_dummies(combined, drop_first=True)
X_train_enc = combined_encoded.iloc[:len(X_train_split), :].copy()
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Randomized search with 25 random combinations instead of all 216
search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_distributions=param_grid,
    n_iter=5,                 # try only 25 random combos
    cv=2,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42
)

search.fit(X_train_enc, y_train_split)

print("Best parameters found:")
print(search.best_params_)
best_model = search.best_estimator_

Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV] END max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time= 1.2min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time= 1.2min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=500; total time= 1.0min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=500; total time= 1.1min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=500; total time= 1.2min
[CV] END max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=500; total time= 1.2min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=500; total time= 1.0min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n

In [114]:
best_model = search.best_estimator_
X_valid_enc = combined_encoded.iloc[len(X_train_split):, :].copy()

In [118]:
y_pred = best_model.predict(X_valid_enc)

In [120]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
r2 = r2_score(y_valid, y_pred)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

MAE: 0.0443
RMSE: 0.0570
R²: 0.8824


In [121]:
test = pd.read_csv("/kaggle/input/playground-series-s5e10/test.csv")

In [122]:
# Combine train+valid+test for consistent one-hot encoding
combined_final = pd.concat([X_train_split, X_valid, test], axis=0)
combined_final_encoded = pd.get_dummies(combined_final, drop_first=True)

# Split back the encoded test portion
X_test_enc = combined_final_encoded.iloc[len(X_train_split) + len(X_valid):, :].copy()

In [126]:
y_test_pred = best_model.predict(X_test_enc)

In [127]:
sample = pd.read_csv("/kaggle/input/playground-series-s5e10/sample_submission.csv")
sample['accident_risk'] = y_test_pred

In [128]:
sample.to_csv("submission.csv", index=False)
print("✅ Submission file saved as submission.csv")

✅ Submission file saved as submission.csv
